MINMAX ALGO

In [1]:
import math

class Node:
    def __init__(self, value=None):
        self.value = value
        self.children = []
        self.minmax_val = None

class MinMax:
    def __init__(self, depth):
        self.depth = depth
    
    def formulate_goal(self, node):
        return "goal found" if node.minmax_val is not None else "not found"
    
    def act(self, node, environment):
        goal_status = self.formulate_goal(node)
        if goal_status == "goal found":
            return f"minmax val for root node: {node.minmax_val}"
        else:
            environment.compute_minmax(node, self.depth)
            return f"minmax val for root node: {node.minmax_val}"

class Environment:
    def __init__(self, tree):
        self.tree = tree
        self.computed_nodes = []
    
    def get_percept(self, node):
        return node
    
    def compute_minmax(self, node, depth, max_player=True):
        if depth == 0 or not node.children:
            self.computed_nodes.append(node.value)
            return node.value
        if max_player:
            value = -math.inf
            for child in node.children:
                child_value = self.compute_minmax(child, depth - 1, False)
                value = max(value, child_value)
        else:
            value = math.inf
            for child in node.children:
                child_value = self.compute_minmax(child, depth - 1, True)
                value = min(value, child_value)
        node.minmax_val = value
        self.computed_nodes.append(node.value)
        return value

def run_agent(agent, environment, start_node):
    percept = environment.get_percept(start_node)
    result = agent.act(percept, environment)
    return result

# Build the sample tree
root = Node('a')
n1 = Node('b')
n2 = Node('c')
root.children = [n1, n2]

n3 = Node('d')
n4 = Node('e')
n5 = Node('f')
n6 = Node('g')
n1.children = [n3, n4]
n2.children = [n5, n6]

n3.children = [Node(2), Node(3)]
n4.children = [Node(5), Node(9)]
n5.children = [Node(0), Node(1)]
n6.children = [Node(7), Node(5)]

# Define depth and run
depth = 3
agent = MinMax(depth)
environment = Environment(root)
print(run_agent(agent, environment, root))
print('Computed Nodes:', environment.computed_nodes)

# Display Minimax values for internal nodes
for node in [root, n1, n2, n3, n4, n5, n6]:
    print(f'{node.value}: {node.minmax_val}')


minmax val for root node: 3
Computed Nodes: [2, 3, 'd', 5, 9, 'e', 'b', 0, 1, 'f', 7, 5, 'g', 'c', 'a']
a: 3
b: 3
c: 1
d: 3
e: 9
f: 1
g: 7


In [2]:
# Alpha-beta pruning saves time by pruning the nides which wont be used in the rogram later
'''alpha: best value of max(-inf)
   beta: best value of min(+inf)'''
import math
class Node:
    def __init__(self,value):
        self.value =value
        self.children = []
        self.minmax_val= None

class AlphaBeta:
    def __init__(self,depth):
        self.depth = depth
    
    def get_goal(self,node):
        return "goal found" if node.minmax_val is not None else "searchingg"

    def act(self,node,environment):
        goal_status = self.get_goal(node)
        if goal_status == "goal found":
            return node.minmax_val
        else:
            return environment.minmax_search(node,self.depth,-math.inf,math.inf,True)

class Environment:
    def __init__(self,tree):
        self.tree = tree
        self.computed_nodes = []

    def get_percept(self,node):
        return node
    
    def minmax_search(self,node,depth,alpha,beta,max_player=True):
        if depth == 0 or not node.children:
            self.computed_nodes.append(node.value)
            return node.value
        if max_player:
            value = -math.inf
            for child in node.children:
                value = max(value,self.minmax_search(child,depth-1,alpha,beta,False))
                alpha = max(alpha,value)
                if beta<=alpha:
                    print("pruned: ",child.value)
                    break
            node.minmax_val = value
            return value
        else:
            value = math.inf
            for child in node.children:
                value = min(value,self.minmax_search(child,depth-1,alpha,beta,True))
                beta = min(beta,value)
                if beta<=alpha:
                    print("pruned; ",child.value)
                    break
            node.minmax_val=value
            return value
def run_agent(agent, environment, start_node):
    percept = environment.get_percept(start_node)
    result = agent.act(percept, environment)
    return result

# Build the sample tree
root = Node('a')
n1 = Node('b')
n2 = Node('c')
root.children = [n1, n2]

n3 = Node('d')
n4 = Node('e')
n5 = Node('f')
n6 = Node('g')
n1.children = [n3, n4]
n2.children = [n5, n6]

n3.children = [Node(2), Node(3)]
n4.children = [Node(5), Node(9)]
n5.children = [Node(0), Node(1)]
n6.children = [Node(7), Node(5)]

# Define depth and run
depth = 3
agent = AlphaBeta(depth)
environment = Environment(root)
print(run_agent(agent, environment, root))
print('Computed Nodes:', environment.computed_nodes)

# Display Minimax values for internal nodes
for node in [root, n1, n2, n3, n4, n5, n6]:
    print(f'{node.value}: {node.minmax_val}')



pruned:  5
pruned;  f
3
Computed Nodes: [2, 3, 5, 0, 1]
a: 3
b: 3
c: 1
d: 3
e: 5
f: 1
g: None


In [ ]:
import copy

EMPTY = '.'
WHITE = 'W'
BLACK = 'B'
ROWS, COLS = 8, 8


def create_board():
    board = [[EMPTY for _ in range(8)] for _ in range(8)]

    def place_pieces(rows, piece_type):
        for r in rows:
            for c in range(8):
                if (r + c) % 2 == 1:
                    board[r][c] = piece_type

    place_pieces(range(0, 3), BLACK)  # Top three rows for BLACK
    place_pieces(range(5, 8), WHITE)  # Bottom three rows for WHITE

    return board



def print_board(board):
    print("  " + " ".join(map(str, range(COLS))))
    for i, row in enumerate(board):
        print(i, " ".join(row))


def get_moves(board, player):
    direction = -1 if player == WHITE else 1
    moves = []
    captures = []

    for r in range(ROWS):
        for c in range(COLS):
            if board[r][c] == player:
                for dc in [-1, 1]:
                    nr, nc = r + direction, c + dc
                    if 0 <= nr < ROWS and 0 <= nc < COLS and board[nr][nc] == EMPTY:
                        moves.append(((r, c), (nr, nc)))
                    # Capture
                    nr2, nc2 = r + 2*direction, c + 2*dc
                    if 0 <= nr2 < ROWS and 0 <= nc2 < COLS:
                        if board[nr][nc] == (BLACK if player == WHITE else WHITE) and board[nr2][nc2] == EMPTY:
                            captures.append(((r, c), (nr2, nc2)))

    return captures if captures else moves


def apply_move(board, move):
    (r1, c1), (r2, c2) = move
    piece = board[r1][c1]
    new_board = copy.deepcopy(board)
    new_board[r1][c1] = EMPTY
    new_board[r2][c2] = piece
    if abs(r2 - r1) == 2:  # Capture
        new_board[(r1 + r2)//2][(c1 + c2)//2] = EMPTY
    return new_board


def evaluate(board):
    white_count = sum(row.count(WHITE) for row in board)
    black_count = sum(row.count(BLACK) for row in board)
    return black_count - white_count


def minimax(board, depth, alpha, beta, maximizing):
    current_player = BLACK if maximizing else WHITE
    moves = get_moves(board, current_player)

    if depth == 0 or not moves:
        return evaluate(board), None

    best_move = None
    if maximizing:
        max_eval = float('-inf')
        for move in moves:
            result = apply_move(board, move)
            eval_score, _ = minimax(result, depth - 1, alpha, beta, False)
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in moves:
            result = apply_move(board, move)
            eval_score, _ = minimax(result, depth - 1, alpha, beta, True)
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval, best_move


def is_game_over(board):
    return not get_moves(board, WHITE) or not get_moves(board, BLACK)


def main():
    board = create_board()
    print_board(board)

    while not is_game_over(board):
        # Human Move
        print("\nYour move (W):")
        moves = get_moves(board, WHITE)
        if not moves:
            print("No moves available. You lose!")
            break
        for i, move in enumerate(moves):
            print(f"{i}: {move}")
        choice = int(input("Select move index: "))
        move = moves[choice]
        board = apply_move(board, move)
        print(f"Player moves: {move[0]} → {move[1]}")
        print_board(board)

        if is_game_over(board):
            print("AI has no moves. You win!")
            break

        # AI Move
        print("\nAI's move (B):")
        _, ai_move = minimax(board, 4, float('-inf'), float('inf'), True)
        if ai_move:
            board = apply_move(board, ai_move)
            print(f"AI moves: {ai_move[0]} → {ai_move[1]}")
            print_board(board)
        else:
            print("AI cannot move. You win!")
            break

    white_pieces = sum(row.count(WHITE) for row in board)
    black_pieces = sum(row.count(BLACK) for row in board)
    if white_pieces == 0:
        print("You lost")
    elif black_pieces == 0:
        print("You won")
    else:
        print("draw or no legal moves left.")

main()


  0 1 2 3 4 5 6 7
0 . B . B . B . B
1 B . B . B . B .
2 . B . B . B . B
3 . . . . . . . .
4 . . . . . . . .
5 W . W . W . W .
6 . W . W . W . W
7 W . W . W . W .

Your move (W):
0: ((5, 0), (4, 1))
1: ((5, 2), (4, 1))
2: ((5, 2), (4, 3))
3: ((5, 4), (4, 3))
4: ((5, 4), (4, 5))
5: ((5, 6), (4, 5))
6: ((5, 6), (4, 7))


In [2]:
import math

def alphabeta(cards, left, right, maximizing_player, alpha, beta):
    if left > right:
        return 0

    if maximizing_player:
        # Max's turn: maximize score
        take_left = cards[left] + alphabeta(cards, left + 1, right, False, alpha, beta)
        take_right = cards[right] + alphabeta(cards, left, right - 1, False, alpha, beta)
        value = max(take_left, take_right)
        alpha = max(alpha, value)
        if beta <= alpha:
            return value
        return value
    else:
        if cards[left] < cards[right]:
            return alphabeta(cards, left + 1, right, True, alpha, beta)
        else:
            return alphabeta(cards, left, right - 1, True, alpha, beta)

def max_turn(cards):
    left_val = cards[0]
    right_val = cards[-1]

    score_if_left = left_val + alphabeta(cards, 1, len(cards)-1, False, -math.inf, math.inf)
    score_if_right = right_val + alphabeta(cards, 0, len(cards)-2, False, -math.inf, math.inf)

    if score_if_left >= score_if_right:
        return cards.pop(0)
    else:
        return cards.pop(-1)

def min_turn(cards):
    if cards[0] < cards[-1]:
        return cards.pop(0)
    else:
        return cards.pop(-1)

def play_game(cards):
    max_score = 0
    min_score = 0
    turn = 'Max'

    while cards:
        print(f"Remaining cards: {cards}")
        if turn == 'Max':
            chosen = max_turn(cards)
            max_score += chosen
            print(f"Max picks: {chosen}")
            turn = 'Min'
        else:
            chosen = min_turn(cards)
            min_score += chosen
            print(f"Min picks: {chosen}")
            turn = 'Max'

    print("\nFinal Scores:")
    print(f"Max's Score: {max_score}")
    print(f"Min's Score: {min_score}")
    if max_score > min_score:
        print("Max wins!")
    elif max_score < min_score:
        print("Min wins!")
    else:
        print("It's a draw!")

cards = [1,3,5,7,9,11]
play_game(cards)


Remaining cards: [1, 3, 5, 7, 9, 11]
Max picks: 11
Remaining cards: [1, 3, 5, 7, 9]
Min picks: 1
Remaining cards: [3, 5, 7, 9]
Max picks: 9
Remaining cards: [3, 5, 7]
Min picks: 3
Remaining cards: [5, 7]
Max picks: 7
Remaining cards: [5]
Min picks: 5

Final Scores:
Max's Score: 27
Min's Score: 9
Max wins!


In [ ]:
import random
import string

GRID_SIZE = 10
SHIP_SIZES = [5, 4]  
def create_board():
    return [['~'] * GRID_SIZE for _ in range(GRID_SIZE)]

def print_board(board, hide_ships=False):
    print("  " + " ".join(string.ascii_uppercase[:GRID_SIZE]))
    for i, row in enumerate(board):
        display_row = []
        for cell in row:
            if hide_ships and cell == 'S':
                display_row.append('~')
            else:
                display_row.append(cell)
        print(f"{i} " + " ".join(display_row))
    print()

def is_valid_placement(board, row, col, size, horizontal):
    for i in range(size):
        r = row
        c = col + i if horizontal else col
        if r >= GRID_SIZE or c >= GRID_SIZE or board[r][c] != '~':
            return False
    return True

def place_ship(board, size):
    placed = False
    while not placed:
        horizontal = random.choice([True, False])
        row = random.randint(0, GRID_SIZE - 1)
        col = random.randint(0, GRID_SIZE - size if horizontal else GRID_SIZE - 1)
        if is_valid_placement(board, row, col, size, horizontal):
            for i in range(size):
                r = row
                c = col + i if horizontal else col
                board[r][c] = 'S'
            placed = True

def parse_coordinate(coord):
    try:
        col = string.ascii_uppercase.index(coord[0].upper())
        row = int(coord[1:])
        if 0 <= row < GRID_SIZE and 0 <= col < GRID_SIZE:
            return row, col
    except:
        pass
    return None

def player_turn(ai_board, display_board):
    while True:
        coord = input("Enter coordinate to fire (e.g., B4): ")
        pos = parse_coordinate(coord)
        if not pos:
            print("Invalid coordinate. Try again.")
            continue
        row, col = pos
        if display_board[row][col] != '~':
            print("Already targeted. Try again.")
            continue
        if ai_board[row][col] == 'S':
            print("Hit!")
            display_board[row][col] = 'X'
            ai_board[row][col] = 'X'
        else:
            print("Miss.")
            display_board[row][col] = 'O'
        break

def ai_turn(player_board, ai_memory):
    # Hunt mode: random search or targeting neighbors
    targets = ai_memory['targets']
    if targets:
        row, col = targets.pop()
    else:
        while True:
            row = random.randint(0, GRID_SIZE - 1)
            col = random.randint(0, GRID_SIZE - 1)
            if player_board[row][col] in ('~', 'S'):
                break
    print(f"AI fires at {string.ascii_uppercase[col]}{row}")
    if player_board[row][col] == 'S':
        print("AI hit your ship!")
        player_board[row][col] = 'X'
        # Add adjacent cells to AI targets
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            r, c = row + dr, col + dc
            if 0 <= r < GRID_SIZE and 0 <= c < GRID_SIZE:
                if player_board[r][c] in ('~', 'S'):
                    ai_memory['targets'].append((r, c))
    else:
        print("AI missed.")
        player_board[row][col] = 'O'

def all_ships_sunk(board):
    return all(cell != 'S' for row in board for cell in row)

def main():
    player_board = create_board()
    ai_board = create_board()
    player_view = create_board()
    ai_memory = {'targets': []}

    print("Placing your ships randomly...")
    for size in SHIP_SIZES:
        place_ship(player_board, size)

    print("Placing AI ships...")
    for size in SHIP_SIZES:
        place_ship(ai_board, size)

    while True:
        print_board(player_view, hide_ships=False)
        player_turn(ai_board, player_view)
        if all_ships_sunk(ai_board):
            print("Congratulations!")
            break
        ai_turn(player_board, ai_memory)
        if all_ships_sunk(player_board):
            print("You lost")
            break

if __name__ == '__main__':
    main()


Placing your ships randomly...
Placing AI ships...
  A B C D E F G H I J
0 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
1 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
2 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
3 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
4 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
5 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
6 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
7 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
8 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
9 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

